In [1]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
# from huggingface_hub import notebook_login

# notebook_login()


In [3]:

!nvidia-smi

Thu Aug 17 10:16:49 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:AF:00.0 Off |                    0 |
| N/A   67C    P0              32W /  70W |      2MiB / 15360MiB |      7%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
PATH = "/root/.cache/huggingface/hub/models--meta-llama--Llama-2-7b-chat-hf"
model_name = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name,
                                          local_files_only=True
                                          # use_auth_token=True,
                                         )



In [5]:
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             local_files_only=True,
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             temperature=0
                                             # use_auth_token=True,
                                            #  load_in_8bit=True,
                                            #  load_in_4bit=True
                                             )

2023-08-17 10:16:54.783565: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-17 10:16:55.626904: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
!ls -lah /root/.cache/huggingface/hub/models--meta-llama--Llama-2-7b-chat-hf

total 4.0K
drwxr-xr-x 6 root root   65 2023-08-16__17:33:10 .
drwxr-xr-x 3 root root   71 2023-08-16__17:33:10 ..
drwxr-xr-x 2 root root 4.0K 2023-08-16__17:33:10 blobs
drwxr-xr-x 3 root root   54 2023-08-16__15:24:26 .no_exist
drwxr-xr-x 2 root root   18 2023-08-16__15:24:22 refs
drwxr-xr-x 3 root root   54 2023-08-16__15:24:22 snapshots


In [7]:
# Use a pipeline for later
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [8]:

!nvidia-smi

Thu Aug 17 10:17:06 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:AF:00.0 Off |                    0 |
| N/A   65C    P0              32W /  70W |  13095MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [9]:
import json
import textwrap

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<>\n", "\n<>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""



def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT ):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_template

def cut_off_text(text, prompt):
    cutoff_phrase = prompt
    index = text.find(cutoff_phrase)
    if index != -1:
        return text[:index]
    else:
        return text

def remove_substring(string, substring):
    return string.replace(substring, "")



def generate(text):
    prompt = get_prompt(text)
    with torch.autocast('cuda', dtype=torch.bfloat16):
        inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
        outputs = model.generate(**inputs,
                                 max_new_tokens=512,
                                 eos_token_id=tokenizer.eos_token_id,
                                 pad_token_id=tokenizer.eos_token_id,
                                 )
        final_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        final_outputs = cut_off_text(final_outputs, '')
        final_outputs = remove_substring(final_outputs, prompt)

    return final_outputs#, outputs

def parse_text(text):
        wrapped_text = textwrap.fill(text, width=100)
        print(wrapped_text +'\n\n')
        # return assistant_text

In [10]:
def llama_generate(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
    outputs = model.generate(**inputs,
                            max_new_tokens=512,
                            eos_token_id=tokenizer.eos_token_id,
                            pad_token_id=tokenizer.eos_token_id,
                            )
    final_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    
    # final_outputs = cut_off_text(final_outputs, '')
    final_outputs = remove_substring(final_outputs, prompt)
    return final_outputs

In [12]:
# !set PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:6144

In [14]:
# !set TOKENIZERS_PARALLELISM=true

## A1: To navigate to the RMA request page, click on "Warranty Claims" in the left menu, then click on the "+Add" button.

In [15]:

prompt1 = """
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

  2 / 6
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  2 wareconn standard operating procedure
SOP
I. This guide will walk you through the process of requesting  RMA  through the
Wareconn Customer Portal.
II. Details on how to fill in the Part Number (PN) and Serial Number (SN)
1.1 RMA request  procedure (Parts return)
1. System login
⚫ Module: https://www.wareconn.com/
⚫ Description:  Login  with account and password

To log in, follow these steps:
Go to https://www.wareconn.com/ and click Log in

Fill in your email and password, then click Login

Click Enter  in the Customer portal3 / 6
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  3 wareconn standard operating procedure
2. Go to  RMA Request  Page
⚫ Module: Customer portal -Warranty Claims -Warranty Claims
⚫ Description: Go to RMA request page

To go to the RMA request page, follow these steps:
Click Warranty Claims  in the left menu, then click +Add

Click the icon shown in the image below
(Select the corresponding SI )4 / 6
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  4 wareconn standard operating procedure
3. Create RMA Request
⚫ Module: Customer portal -Warranty Claims -Warranty Claims -Create request
⚫ Description: Create RMA request order

To create a RMA request, follow these steps:
Click Add Warranty Claims

Then, click Parts Return

  Question: How can I navigate to the RMA request page in the Wareconn Customer Portal?
  Answer:
"""

In [18]:
%%time
out1 = llama_generate(prompt1)

CPU times: user 5.66 s, sys: 1.99 s, total: 7.65 s
Wall time: 7.66 s


In [19]:
print(out1)

You can navigate to the RMA request page in the Wareconn Customer Portal by following these steps:
1. Go to the Customer portal and log in with your email and password.
2. Click on the Warranty Claims module in the left menu.
3. Click on the +Add button next to Warranty Claims.
4. Select the corresponding SI (System Interface) and click Enter.
5. Click on the Create Request button to open the RMA request page.


## Ans2: The Reminder field shows whether the RMA request is approvable (acceptable) for the warranty provider or not.

In [20]:
prompt2 = """
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

  6 / 6
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  6 wareconn standard operating procedure
5. Submit the RMA  request

⚫ Module: Customer portal -Warranty Claims -Warranty Claims -Submit
⚫ Description: Submit the RMA request

The Reminder field will show you whether this RMA request is approvable
(acceptable)  for warranty provider  or not.
Please remember to fill in WAL# in Customer No. field
Lastly, click Submit  in the top right corner to finish the  RMA requesting process2 / 6
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  2 wareconn standard operating procedure
SOP
I. This guide will walk you through the process of requesting  RMA  through the
Wareconn Customer Portal.
II. Details on how to fill in the Part Number (PN) and Serial Number (SN)
1.1 RMA request  procedure (Parts return)
1. System login
⚫ Module: https://www.wareconn.com/
⚫ Description:  Login  with account and password

To log in, follow these steps:
Go to https://www.wareconn.com/ and click Log in

Fill in your email and password, then click Login

Click Enter  in the Customer portal4 / 6
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  4 wareconn standard operating procedure
3. Create RMA Request
⚫ Module: Customer portal -Warranty Claims -Warranty Claims -Create request
⚫ Description: Create RMA request order

To create a RMA request, follow these steps:
Click Add Warranty Claims

Then, click Parts Return

  Question: What does the Reminder field indicate when submitting an RMA request?
  Answer:
"""

In [21]:
%%time
print(llama_generate(prompt2))

The Reminder field indicates whether the RMA request is approvable (acceptable) for warranty provider or not.
CPU times: user 1.56 s, sys: 607 ms, total: 2.17 s
Wall time: 2.17 s


Ans3: When submitting an RMA request, you should fill in the WAL# (Warranty Authorization Number) in the Customer No. field.

In [22]:
prompt3 = """
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

  6 / 6
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  6 wareconn standard operating procedure
5. Submit the RMA  request

⚫ Module: Customer portal -Warranty Claims -Warranty Claims -Submit
⚫ Description: Submit the RMA request

The Reminder field will show you whether this RMA request is approvable
(acceptable)  for warranty provider  or not.
Please remember to fill in WAL# in Customer No. field
Lastly, click Submit  in the top right corner to finish the  RMA requesting process2 / 6
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  2 wareconn standard operating procedure
SOP
I. This guide will walk you through the process of requesting  RMA  through the
Wareconn Customer Portal.
II. Details on how to fill in the Part Number (PN) and Serial Number (SN)
1.1 RMA request  procedure (Parts return)
1. System login
⚫ Module: https://www.wareconn.com/
⚫ Description:  Login  with account and password

To log in, follow these steps:
Go to https://www.wareconn.com/ and click Log in

Fill in your email and password, then click Login

Click Enter  in the Customer portal4 / 6
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  4 wareconn standard operating procedure
3. Create RMA Request
⚫ Module: Customer portal -Warranty Claims -Warranty Claims -Create request
⚫ Description: Create RMA request order

To create a RMA request, follow these steps:
Click Add Warranty Claims

Then, click Parts Return

  Question: What information should be filled in the Customer No. field when submitting an RMA request?
  Answer:
"""

In [28]:
%%time
print(llama_generate(prompt3))

The Customer No. field should be filled in with the WAL# (Warranty Approval Letter) provided by the warranty provider.
CPU times: user 1.89 s, sys: 688 ms, total: 2.57 s
Wall time: 2.58 s


## Ans4: To fill in the request information for an RMA, follow these steps:

- Fill in the Product SN (Serial Number), Product PN (Part Number), and defective reason.
- Select the Rack PN (Part Number) by clicking on "Please select Parts" and choose the desired quantity.
- Click on "Confirm" to proceed.
- Optionally, fill in the Parts SN (Serial Number) for the failure part (not required).
- Click on "Save" to save the information.

In [29]:
prompt4 = """
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

  2 / 6
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  2 wareconn standard operating procedure
SOP
I. This guide will walk you through the process of requesting  RMA  through the
Wareconn Customer Portal.
II. Details on how to fill in the Part Number (PN) and Serial Number (SN)
1.1 RMA request  procedure (Parts return)
1. System login
⚫ Module: https://www.wareconn.com/
⚫ Description:  Login  with account and password

To log in, follow these steps:
Go to https://www.wareconn.com/ and click Log in

Fill in your email and password, then click Login

Click Enter  in the Customer portal4 / 6
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  4 wareconn standard operating procedure
3. Create RMA Request
⚫ Module: Customer portal -Warranty Claims -Warranty Claims -Create request
⚫ Description: Create RMA request order

To create a RMA request, follow these steps:
Click Add Warranty Claims

Then, click Parts Return6 / 6
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  6 wareconn standard operating procedure
5. Submit the RMA  request

⚫ Module: Customer portal -Warranty Claims -Warranty Claims -Submit
⚫ Description: Submit the RMA request

The Reminder field will show you whether this RMA request is approvable
(acceptable)  for warranty provider  or not.
Please remember to fill in WAL# in Customer No. field
Lastly, click Submit  in the top right corner to finish the  RMA requesting process

  Question: What steps should be followed when filling in the request information for an RMA?
  Answer:
"""

In [30]:
%%time
print(llama_generate(prompt4))

I don't know.
CPU times: user 657 ms, sys: 342 ms, total: 999 ms
Wall time: 998 ms


## Ans5: No, this document explicitly states that it should not be used for other purposes. Its sole intention is to introduce Wareconn functions and procedures.

In [31]:
prompt5 = """
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

  1 / 6
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  1 wareconn standard operating procedure






wareconn standard operating procedure
RMA request  SOP

Version :
Version  Date  Editor  Description
v 1.0  2023/03/16 Eric Sun First draft (English version)


Content
SOP ................................ ................................ ................................ ................................  2
1.1 RMA request procedure (Parts return)  ................................ ...............  24 / 6
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  4 wareconn standard operating procedure
3. Create RMA Request
⚫ Module: Customer portal -Warranty Claims -Warranty Claims -Create request
⚫ Description: Create RMA request order

To create a RMA request, follow these steps:
Click Add Warranty Claims

Then, click Parts Return3 / 6
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  3 wareconn standard operating procedure
2. Go to  RMA Request  Page
⚫ Module: Customer portal -Warranty Claims -Warranty Claims
⚫ Description: Go to RMA request page

To go to the RMA request page, follow these steps:
Click Warranty Claims  in the left menu, then click +Add

Click the icon shown in the image below
(Select the corresponding SI )

  Question: Can this document be used for purposes other than introducing Wareconn functions and procedures?
  Answer:
"""

In [32]:
%%time
print(llama_generate(prompt5))

I don't know.
CPU times: user 590 ms, sys: 291 ms, total: 881 ms
Wall time: 880 ms
